In [2]:
import pandas as pd
import numpy as np

In [2]:
grid = pd.read_csv('./Data/final_game_user_grid_06_07_19.csv')

In [3]:
grid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1387492 entries, 0 to 1387491
Data columns (total 7 columns):
user_name       1387492 non-null object
game_name       1385813 non-null object
game_genres     1383564 non-null object
language        1387490 non-null object
started_at      1387492 non-null object
viewer_count    1387492 non-null int64
max             1387492 non-null int64
dtypes: int64(2), object(5)
memory usage: 74.1+ MB


In [4]:
grid = grid.dropna()

In [5]:
grid = grid.dropna(how='any',axis=0) 

In [6]:
x = grid
x['game_genres'][x['game_genres'].str.contains('NONE') == True] = 'Other'
x['game_genres'][x['game_genres'].isnull()] = 'Other'
x['game_genres'][x['game_genres'].str.contains('RETROGAMEPLACEHOLDER')] = 'Other'

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [7]:
grid = x

In [15]:
# To begin, we're calculating the metric we'll be using to compare users and games. Since we're focused on the streamers, we don't have the traditional "I like this movie so I will rate it a 4.5 out of 5.0" ratings. Instead we're calculating how many viewers each streamer attracted with a particular game compared to their max viewer potential over the last week. For each user, one game is their ultimate streaming "5 out of 5" benchmark, and all other games they play are compared to that one, and normalized to ratings between 1 and 5. We are also using all the genres for each game to pinpoint how successful various genres and games have been for each streamer.

In [8]:
max_value_username = pd.DataFrame(grid.groupby('user_name')['max'].max().reset_index())

In [9]:
max_val_dict = max_value_username.groupby('user_name')['max'].apply(list).to_dict()

In [10]:
grid['max_game'] = grid['user_name'].map(max_val_dict)

In [11]:
grid['max_game_int'] = grid.max_game.str[0].astype(int)
grid = grid.drop('max_game', axis = 1)

In [12]:
grid['score'] = grid['max']/grid['max_game_int']

In [13]:
from sklearn.preprocessing import minmax_scale
grid['scaled_score'] = minmax_scale(grid['score'], feature_range=(1, 5))

In [14]:
grid = grid.dropna()

In [13]:
# now we have a listing for each user pairing them with each game they play, what genre it belongs to, and how many people watched them play each game compared to the max viewers they ever got for a stream during the week we examined.

In [15]:
grid.groupby('user_name')['scaled_score'].count().reset_index().sort_values('scaled_score', ascending=False)[:5]

,user_name,scaled_score
27473,Fonbet_RocketLeague,378
77300,StreamerHouse,375
123375,luke4316live,333
110114,gaules,315
27472,Fonbet_RLH,270


In [16]:
grid.groupby('game_genres')['scaled_score'].count().reset_index().sort_values('scaled_score', ascending=False)[:5]

,game_genres,scaled_score
0,Action,211633
28,Shooter,165931
7,FPS,131286
23,RPG,118595
15,MMORPG,88901


In [17]:
grid.groupby('game_name')['scaled_score'].count().reset_index().sort_values('scaled_score', ascending=False)[:5]

,game_name,scaled_score
128,Grand Theft Auto V,37860
180,Minecraft,35715
243,Rocket League,35043
37,Black Desert Online,33723
97,Escape From Tarkov,33366


In [18]:
min_number_scores = 5
filter_users = grid['user_name'].value_counts() > min_number_scores
filter_users = filter_users[filter_users].index.tolist()

In [19]:
grid_new = grid[(grid['user_name'].isin(filter_users))]
print('The original data frame shape:\t{}'.format(grid.shape))
print('The new data frame shape:\t{}'.format(grid_new.shape))

The original data frame shape:	(1364027, 10)
The new data frame shape:	(1137673, 10)


In [20]:
# After reshaping the grid, we are going to extract the recommendations for game genre, game titles, and games similar to those already rated by the streamers as a three-pronged recommender approach.

In [4]:
import pickle

In [22]:
pickle.dump(grid_new, open("./Data/final_grid_06_07_19.pkl", "wb" ) )

In [5]:
grid_new = pickle.load( open( "./Data/final_grid_06_07_19.pkl", "rb" ) )

In [11]:
# preparing for genre recommendations based on viewership scores:

In [6]:
genres = grid_new.groupby(by = ['user_name', 'game_genres'])['scaled_score'].agg([np.mean])
games = grid_new.groupby(by = ['user_name', 'game_name'])['scaled_score'].agg([np.mean])

In [25]:
genres.head(5)

mean
user_name     game_genres          
00NothingLabs Fighting     2.421053
              Open World   1.578947
              RPG          1.578947
              Shooter      1.578947
00elu00       Action       4.000000

In [26]:
games.head(5)

mean
user_name     game_name                            
00NothingLabs Mortal Kombat 11             2.421053
              Tom Clancy's The Division 2  1.578947
00elu00       Dead by Daylight             4.666667
              Deathgarden                  2.000000
01joga        PUBG MOBILE                  3.750000

In [7]:
genres.columns = genres.columns.map(''.join)
games.columns = games.columns.map(''.join)

In [8]:
genres = genres.reset_index()
games = games.reset_index()

In [29]:
genres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181215 entries, 0 to 181214
Data columns (total 3 columns):
user_name      181215 non-null object
game_genres    181215 non-null object
mean           181215 non-null float64
dtypes: float64(1), object(2)
memory usage: 4.1+ MB


In [30]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93606 entries, 0 to 93605
Data columns (total 3 columns):
user_name    93606 non-null object
game_name    93606 non-null object
mean         93606 non-null float64
dtypes: float64(1), object(2)
memory usage: 2.1+ MB


In [31]:
genres.head(5)

,user_name,game_genres,mean
0,00NothingLabs,Fighting,2.421053
1,00NothingLabs,Open World,1.578947
2,00NothingLabs,RPG,1.578947
3,00NothingLabs,Shooter,1.578947
4,00elu00,Action,4.000000


In [32]:
games.head(5)

,user_name,game_name,mean
0,00NothingLabs,Mortal Kombat 11,2.421053
1,00NothingLabs,Tom Clancy's The Division 2,1.578947
2,00elu00,Dead by Daylight,4.666667
3,00elu00,Deathgarden,2.000000
4,01joga,PUBG MOBILE,3.750000


In [33]:
#using Surprise to predict genres/games for a streamer based on their existing genre ratings:

In [9]:
import surprise
from surprise import Dataset, accuracy, Reader, NMF, NormalPredictor, BaselineOnly, CoClustering, SlopeOne, SVD, KNNBaseline
from surprise.model_selection import GridSearchCV, cross_validate, train_test_split

In [10]:
reader = Reader(rating_scale=(1, 5))
genre_data = Dataset.load_from_df(genres[['user_name', 'game_genres', 'mean']], reader)

In [11]:
game_data = Dataset.load_from_df(games[['user_name', 'game_name', 'mean']], reader)

Now it is time to select the algorithms we will use to generate recommendations for our streamers based on their existing behavior and also on how similar the games they already stream are to those they do not yet stream. 

The first step is to evaluate various algorithms available in Surprise to see which produce the lowest RMSE and are computationally friendly so we can use them in a dynamic app and recommend things on the fly. We perform cross validation on all fast algorithms to see which performs best with either of our datasets, genres and games.

In [12]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SlopeOne(), NormalPredictor(), BaselineOnly(), NMF(), CoClustering(), SVD()]:


    # Perform cross validation
    results = cross_validate(algorithm, genre_data, measures=['RMSE'], return_train_measures=True, cv=5, verbose=True)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')  

Evaluating RMSE of algorithm SlopeOne on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.5829  0.5851  0.5824  0.5794  0.5870  0.5833  0.0026  
RMSE (trainset)   0.3760  0.3753  0.3760  0.3765  0.3752  0.3758  0.0005  
Fit time          0.52    0.56    0.56    0.62    0.57    0.57    0.03    
Test time         0.39    0.35    0.36    0.36    0.43    0.38    0.03    
Evaluating RMSE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1031  1.1006  1.1056  1.1024  1.1099  1.1043  0.0032  
RMSE (trainset)   1.1030  1.1078  1.1075  1.1037  1.1042  1.1052  0.0020  
Fit time          0.32    0.26    0.21    0.21    0.22    0.25    0.04    
Test time         0.39    0.20    0.28    0.30    0.20    0.28    0.07    
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating 

,test_rmse,train_rmse,fit_time,test_time
Algorithm,,,,
SlopeOne,0.583346,0.375794,0.565539,0.378328
NMF,0.663177,0.143120,11.781459,0.232531
SVD,0.684103,0.548758,8.086810,0.270559
BaselineOnly,0.736001,0.695091,0.460336,0.252608
CoClustering,0.759168,0.628109,7.972136,0.194265
NormalPredictor,1.104331,1.105233,0.245939,0.276477


For genre data, the SlopeOne seems to do the best algorithm. To drastically reduce overfitting, improve performance and ease implementation, the Slope One family of easily implemented Item-based Rating-Based collaborative filtering algorithms was proposed. Essentially, instead of using linear regression from one item's ratings to another item's ratings ( {\displaystyle f(x)=ax+b} f(x)=ax+b), it uses a simpler form of regression with a single free parameter ( {\displaystyle f(x)=x+b} f(x)=x+b). The free parameter is then simply the average difference between the two items' ratings. It was shown to be much more accurate than linear regression in some instances, and it takes half the storage or less.

In [13]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SlopeOne(), NormalPredictor(), BaselineOnly(), NMF(), CoClustering(), SVD()]:


    # Perform cross validation
    results = cross_validate(algorithm, game_data, measures=['RMSE'], return_train_measures=True, cv=5, verbose=True)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')  

Evaluating RMSE of algorithm SlopeOne on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9728  0.9633  0.9585  0.9623  0.9628  0.9639  0.0047  
RMSE (trainset)   0.3340  0.3371  0.3367  0.3366  0.3368  0.3363  0.0011  
Fit time          0.46    0.51    0.50    0.53    0.57    0.51    0.03    
Test time         0.13    0.18    0.13    0.13    0.18    0.15    0.02    
Evaluating RMSE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1564  1.1550  1.1517  1.1554  1.1559  1.1549  0.0017  
RMSE (trainset)   1.1569  1.1535  1.1571  1.1519  1.1501  1.1539  0.0027  
Fit time          0.09    0.11    0.13    0.11    0.11    0.11    0.01    
Test time         0.10    0.10    0.13    0.11    0.10    0.11    0.01    
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating 

,test_rmse,train_rmse,fit_time,test_time
Algorithm,,,,
BaselineOnly,0.798761,0.748669,0.267839,0.127457
SVD,0.803507,0.599217,4.330534,0.147899
SlopeOne,0.963917,0.336254,0.514620,0.151804
NMF,0.977529,0.102057,7.623752,0.124938
CoClustering,1.031269,0.676456,6.193704,0.123336
NormalPredictor,1.154859,1.153893,0.108619,0.106389


For the game data, we are going to use the BaselineOnly algorithm which produces the lowest test RMSE.

Based on the results of the algorithm selector, we will proceed with SlopeOne for genre predictions and with the Baseline algorithm for game predictions. More information about SlopeOne can be found here: https://arxiv.org/abs/cs/0702144

Next we perform cross-validation to see how our chosen algorithms perform:

In [56]:
algo_genre = SlopeOne()
cross_validate(algo_genre, genre_data, measures=['RMSE'], cv=7, verbose=True)

Evaluating RMSE of algorithm SlopeOne on 7 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Mean    Std     
RMSE (testset)    0.5882  0.5934  0.5916  0.5873  0.5844  0.5948  0.5797  0.5885  0.0049  
Fit time          0.54    0.69    0.70    0.77    0.73    0.69    0.69    0.69    0.07    
Test time         0.31    0.24    0.22    0.24    0.22    0.23    0.24    0.24    0.03    


{'test_rmse': array([0.58820224, 0.59338724, 0.59159793, 0.58734185, 0.58436884,
        0.59476135, 0.57966551]),
 'fit_time': (0.5381171703338623,
  0.6905372142791748,
  0.7029690742492676,
  0.7682769298553467,
  0.7282760143280029,
  0.6883859634399414,
  0.6933460235595703),
 'test_time': (0.31470394134521484,
  0.24323701858520508,
  0.2221078872680664,
  0.2357029914855957,
  0.21745991706848145,
  0.23285627365112305,
  0.23707890510559082)}

In [57]:
bsl_options = {'method': 'als',
               'n_epochs': 10,
               'reg_u': 12,
               'reg_i': 5
               }
algo_games = BaselineOnly(bsl_options=bsl_options)
cross_validate(algo_games, game_data, measures=['RMSE'], cv=7, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE of algorithm BaselineOnly on 7 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Mean    Std     
RMSE (testset)    0.8005  0.8045  0.8100  0.7963  0.7992  0.7942  0.7925  0.7996  0.0056  
Fit time          0.27    0.38    0.30    0.29    0.41    0.31    0.27    0.32    0.05    
Test time         0.07    0.06    0.06    0.07    0.08    0.14    0.07    0.08    0.02    


{'test_rmse': array([0.80052949, 0.80450176, 0.80995651, 0.79629228, 0.79922829,
        0.79415421, 0.7925107 ]),
 'fit_time': (0.2749178409576416,
  0.37617993354797363,
  0.3001220226287842,
  0.28711390495300293,
  0.41249704360961914,
  0.31096720695495605,
  0.2670562267303467),
 'test_time': (0.06943488121032715,
  0.059584856033325195,
  0.06414794921875,
  0.0724940299987793,
  0.07842302322387695,
  0.13653993606567383,
  0.06610488891601562)}

Now we split the data into train and test sets to produce the predictions for genres and games streamers would like. Our RMSE is low for both algorithms, lower for the genres which makes sense since each genre includes many games which produces more observations for each genres than for each individual game.

In [54]:
genre_trainset, genre_testset = train_test_split(genre_data, test_size=0.25)
algo_genre = SlopeOne()
genre_predictions = algo_genre.fit(genre_trainset).test(genre_testset)
accuracy.rmse(genre_predictions)

RMSE: 0.6224


0.6224159298329438

In [58]:
game_trainset, game_testset = train_test_split(game_data, test_size=0.25)
algo_games = BaselineOnly(bsl_options=bsl_options)
game_predictions = algo_games.fit(game_trainset).test(game_testset)
accuracy.rmse(game_predictions)

Estimating biases using als...
RMSE: 0.7994


0.7994108918220522

Saving the models and predictions to use in the predictor app:

In [48]:
pickle.dump(algo_genre, open("./Data/SlopeOne_genre_model.pkl", "wb" ) )
algo_genre = pickle.load( open( "./Data/SlopeOne_genre_model.pkl", "rb" ) )

In [49]:
pickle.dump(algo_games, open("./Data/BaselineOnly_game_model.pkl", "wb" ) )
algo_games = pickle.load( open( "./Data/BaselineOnly_game_model.pkl", "rb" ) )

In [50]:
pickle.dump(genre_predictions, open("./Data/SlopeOne_genre_model_predictions.pkl", "wb" ) )
genre_predictions = pickle.load( open( "./Data/SlopeOne_genre_model_predictions.pkl", "rb" ) )

In [51]:
pickle.dump(game_predictions, open("./Data/BaselineOnly_game_model_predictions.pkl", "wb" ) )
game_predictions = pickle.load( open( "./Data/BaselineOnly_game_model_predictions.pkl", "rb" ) )

In [52]:
pickle.dump(genres, open("./Data/genres.pkl", "wb" ) )
genres = pickle.load( open( "./Data/genres.pkl", "rb" ) )

In [53]:
pickle.dump(games, open("./Data/games.pkl", "wb" ) )
games = pickle.load( open( "./Data/games.pkl", "rb" ) )

Taking the inputs from the user:

In [41]:
streamer_name = input('What is your streamer name? ')
streamer_genres = list(input ('Which game genres do you currently stream? ').split(', '))
streamer_games = list(input ('Which games do you currently steam? ').split(', '))

What is your streamer name? Ninja
Which game genres do you currently stream? Action
Which games do you currently steam? Fortnite


In [42]:
streamer_name, streamer_genres, streamer_games

('Ninja', ['Action'], ['Fortnite'])

Making a list of streamers' current genres and games by combining any information we already have in our dataset and their own inputs into the app:

In [43]:
genres.head()

,user_name,game_genres,mean
0,00NothingLabs,Fighting,2.421053
1,00NothingLabs,Open World,1.578947
2,00NothingLabs,RPG,1.578947
3,00NothingLabs,Shooter,1.578947
4,00elu00,Action,4.000000


In [44]:
def display_current_genres(streamer_name):
    user_genres = list(genres[genres['user_name']==streamer_name]['game_genres'])
    return user_genres
recorder_genres_list = display_current_genres(streamer_name)
full_genres = set(recorder_genres_list + streamer_genres)
full_genres

{'Action', 'IRL', 'MOBA', 'Shooter'}

In [45]:
def display_current_games(streamer_name):
    user_games = list(games[games['user_name']==streamer_name]['game_name'])
    return user_games
recorder_games_list = display_current_games(streamer_name)
full_games = set(recorder_games_list + streamer_games)
full_games

{'Brawl Stars', 'Fortnite', 'Just Chatting'}

####  Predicting Genres for Streamers (user-based similarities) ####

In [46]:
from collections import defaultdict

In [47]:
def get_top_n(predictions, n=10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for the user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

Setting the inner ids for genres and games to be able to identify them in Surprise nomenclature. The results are all the genres and games the user has not had exposure to:

In [48]:
iids_genre = genres['game_genres'].unique()
iids_genre_to_predict = np.setdiff1d(iids_genre, full_genres)

In [49]:
iids_game = games['game_name'].unique()
iids_game_to_predict = np.setdiff1d(iids_game, full_games)

In [50]:
iids_genre_to_predict

array(['Action', 'Adventure Game', 'Card & Board Game', 'Compilation',
       'Creative', 'Driving/Racing Game', 'Educational Game', 'FPS',
       'Fighting', 'Flight Simulator', 'Gambling Game', 'Hidden Objects',
       'Horror', 'IRL', 'Indie Game', 'MMORPG', 'MOBA', 'Metroidvania',
       'Open World', 'Other', 'Platformer', 'Point and Click', 'Puzzle',
       'RPG', 'RTS', 'Rhythm & Music Game', 'Roguelike', 'Series: Souls',
       'Shooter', 'Simulation', 'Sports Game', 'Stealth', 'Strategy',
       'Survival', 'Visual Novel'], dtype=object)

In [51]:
iids_game_to_predict

array(['7 Days to Die', 'A Hat in Time', 'A Plague Tale: Innocence',
       'A Way Out', 'ARK', 'ASMR', 'ATLAS', 'Agar.io',
       'Age of Empires II', 'Age of Mythology', 'Albion Online',
       'Alien: Isolation', 'Always On', 'American Truck Simulator',
       'Anno 1800', 'Apex Legends', 'Arena of Valor', 'Arma 3', 'Art',
       "Assassin's Creed II", "Assassin's Creed III",
       "Assassin's Creed Odyssey", 'Assetto Corsa Competizione',
       'Astroneer', 'Auto Chess', 'BaBa Is You', 'Bad North',
       'Banjo-Kazooie', 'Barotrauma', 'Battalion 1944', 'Battle Brothers',
       'Battlefield 1', 'Battlefield 4', 'Battlefield V', 'Beat Saber',
       'BioShock', 'Black Desert Online', 'Blackjack', 'Blood & Truth',
       'Bloodborne', 'Borderlands 2', 'Brawl Stars', 'Brawlhalla',
       'Call of Duty: Black Ops 4', 'Call of Duty: Black Ops II',
       'Call of Duty: Black Ops III', 'Call of Duty: Modern Warfare 3',
       'Call of Duty: Modern Warfare Remastered', 'Call of Duty: WW

Making a personal testset for the user, populating same base rating as true since the actuals are not known and we are trying to predict the expected rating:

In [222]:
genre_testset_personal = [[streamer_name, iid, 0.] for iid in iids_genre_to_predict]
game_testset_personal = [[streamer_name, iid, 0.] for iid in iids_game_to_predict]

Producing a list of predictions based on the inputs:

In [225]:
personal_genre_predictions = algo_genre.test(genre_testset_personal)
personal_game_predictions = algo_games.test(game_testset_personal)

In [234]:
personal_genre_df = pd.DataFrame(personal_genre_predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])    

In [233]:
personal_genre_pred = personal_genre_df[['iid', 'est']]

In [235]:
personal_game_df = pd.DataFrame(personal_game_predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])    

In [236]:
personal_game_pred = personal_genre_df[['iid', 'est']]

In [237]:
top_n_genres = get_top_n(genre_predictions)
top_n_games = get_top_n(game_predictions)

In [238]:
top_n_genres

defaultdict(list,
            {'Ninja': [('Hidden Objects', 3.3662984761001553),
              ('Gambling Game', 2.485279957581637),
              ('Driving/Racing Game', 2.074400611260335),
              ('Visual Novel', 2.0662984761001555),
              ('Flight Simulator', 2.032965142766822),
              ('Creative', 2.0244588217737034),
              ('Simulation', 2.002190288611624),
              ('Sports Game', 1.9230060172042245),
              ('Other', 1.9129250126203088),
              ('Rhythm & Music Game', 1.9057413327389472)]})

In [239]:
top_n_games

defaultdict(list,
            {'Ninja': [("Don't Starve Together", 3.724568448679777),
              ('Visage', 3.715993292343966),
              ('Hollow Knight', 3.6960702010365023),
              ('The Jackbox Party Pack 3', 3.6805546202429182),
              ('Minecraft', 3.6799964801474587),
              ('The Elder Scrolls IV: Oblivion', 3.674633065883794),
              ('Silent Hill', 3.6573610845932816),
              ('Rocket League', 3.646182971664347),
              ('Diablo', 3.6426478761692964),
              ('Trials Rising', 3.639944942411428)]})

In [240]:
for uid, user_ratings in top_n_genres.items():
    print('For ' + uid + ', the recommended genres are:'+ str([iid for (iid, _) in user_ratings]))
genre_user_based_list = [iid for (iid, _) in user_ratings]

For Ninja, the recommended genres are:['Hidden Objects', 'Gambling Game', 'Driving/Racing Game', 'Visual Novel', 'Flight Simulator', 'Creative', 'Simulation', 'Sports Game', 'Other', 'Rhythm & Music Game']


In [241]:
for uid, user_ratings in top_n_games.items():
    print('For ' + uid + ', the recommended games are:'+ str([iid for (iid, _) in user_ratings]))

For Ninja, the recommended games are:["Don't Starve Together", 'Visage', 'Hollow Knight', 'The Jackbox Party Pack 3', 'Minecraft', 'The Elder Scrolls IV: Oblivion', 'Silent Hill', 'Rocket League', 'Diablo', 'Trials Rising']


In [242]:
game_user_based_list = [iid for (iid, _) in user_ratings]
game_user_based_list

["Don't Starve Together",
 'Visage',
 'Hollow Knight',
 'The Jackbox Party Pack 3',
 'Minecraft',
 'The Elder Scrolls IV: Oblivion',
 'Silent Hill',
 'Rocket League',
 'Diablo',
 'Trials Rising']

#### Predicting Similar Genres Based on Current Genre/Game (item-based similarity) #### 

In [59]:
from surprise import KNNBaseline
from surprise import Dataset
from surprise import get_dataset_dir

In [60]:
genre_group = grid_new.groupby(by = ['game_genres', 'user_name'])['scaled_score'].agg([np.mean])
game_group = grid_new.groupby(by = ['game_name', 'user_name'])['scaled_score'].agg([np.mean])

In [61]:
genre_group = genre_group.reset_index()
game_group = game_group.reset_index()

In [57]:
pickle.dump(genre_group, open("./Data/genre_group.pkl", "wb" ) )
genre_group = pickle.load( open( "./Data/genre_group.pkl", "rb" ) )

pickle.dump(game_group, open("./Data/game_group.pkl", "wb" ) )
game_group = pickle.load( open( "./Data/game_group.pkl", "rb" ) )

In [27]:
genre_group = pickle.load( open( "./Data/genre_group.pkl", "rb" ) )
reader = Reader(rating_scale=(1, 5))
genre_data = Dataset.load_from_df(genre_group[['user_name', 'game_genres', 'mean']], reader)

game_group = pickle.load( open( "./Data/game_group.pkl", "rb" ) )
reader = Reader(rating_scale=(1, 5))
game_data = Dataset.load_from_df(game_group[['user_name', 'game_name', 'mean']], reader)

We're using the KNNBaseline algorithm to choose the genres and games with the highest similarity to those already streamed, as it was the fastest of the KNN algorithms available and can be quickly used in the app to make on the spot predictions. We now perform the grid search to find the optimal parameters and then use those algorithms on our data.

In [158]:
import random

from surprise.model_selection import GridSearchCV


# Load the full dataset.
data = genre_data
raw_ratings = data.raw_ratings

# shuffle ratings if you want
random.shuffle(raw_ratings)

# A = 90% of the data, B = 10% of the data
threshold = int(.9 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]

data.raw_ratings = A_raw_ratings  # data is now the set A

# Select your best algo with grid search.
print('Grid Search...')
param_grid = {'bsl_options': {'method': ['als', 'sgd'],
                              'reg': [1, 2]},
              'k': [2,3],
              'sim_options': {'name': ['msd', 'cosine'],
                              'min_support': [1, 5],
                              'user_based': [False]}
              }
grid_search = GridSearchCV(KNNBaseline, param_grid, measures=['rmse'], cv=3)
grid_search.fit(data)

KNN_genre_algo = grid_search.best_estimator['rmse']

# retrain on the whole set A
trainset = data.build_full_trainset()
KNN_genre_algo.fit(trainset)

# Compute biased accuracy on A
predictions = KNN_genre_algo.test(trainset.build_testset())
print('Biased accuracy on A,', end='   ')
accuracy.rmse(predictions)

# Compute unbiased accuracy on B
testset = data.construct_testset(B_raw_ratings)  # testset is now the set B
genre_predictions = KNN_genre_algo.test(testset)
print('Unbiased accuracy on B,', end=' ')
accuracy.rmse(genre_predictions)


Grid Search...
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing

Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the m

0.5817957159484195

In [75]:
import random

from surprise.model_selection import GridSearchCV


# Load the full dataset.
data = game_data
raw_ratings = data.raw_ratings

# shuffle ratings if you want
random.shuffle(raw_ratings)

# A = 90% of the data, B = 10% of the data
threshold = int(.9 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]

data.raw_ratings = A_raw_ratings  # data is now the set A

# Select your best algo with grid search.
print('Grid Search...')
param_grid = {'bsl_options': {'method': ['als', 'sgd'],
                              'reg': [1, 2]},
              'k': [2, 3],
              'sim_options': {'name': ['msd', 'cosine'],
                              'min_support': [1, 5],
                              'user_based': [False]}
              }
grid_search = GridSearchCV(KNNBaseline, param_grid, measures=['rmse'], cv=3)
grid_search.fit(data)

KNN_game_algo = grid_search.best_estimator['rmse']

# retrain on the whole set A
trainset = data.build_full_trainset()
KNN_game_algo.fit(trainset)

# Compute biased accuracy on A
game_predictions = KNN_game_algo.test(trainset.build_testset())
print('Biased accuracy on A,', end='   ')
accuracy.rmse(game_predictions)

# Compute unbiased accuracy on B
testset = data.construct_testset(B_raw_ratings)  # testset is now the set B
game_predictions = KNN_game_algo.test(testset)
print('Unbiased accuracy on B,', end=' ')
accuracy.rmse(game_predictions)


Grid Search...
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing

Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the m

0.8911290992197087

Now we're going to use the best estimator from the grid search to find the nearest neighbors of the genres and games we received as inputs from the streamers:

In [184]:
genre_trainset = genre_data.build_full_trainset()
genre_testset = genre_trainset.build_anti_testset()
final_genre_predictions = KNN_genre_algo.fit(genre_trainset).test(genre_testset)
accuracy.rmse(final_genre_predictions)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.6572


0.6572131030195313

In [185]:
game_trainset = game_data.build_full_trainset()
game_testset = game_trainset.build_anti_testset()
final_game_predictions = KNN_game_algo.fit(game_trainset).test(game_testset)
accuracy.rmse(final_game_predictions)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.3009


0.3008986325312816

To be able to evaluate the closest neighbors of items, we're going to make a list of every genre and game that needs to be evaluated and predict how similar they are going to be those already streamed by our target person:

In [186]:
#produce the list of genres/games needed to be evaluated by converting all the items to their inner ids used by Surprise

genre_inner_id_list = []
for genre in full_genres:
    inner = KNN_genre_algo.trainset.to_inner_iid(genre)
    genre_inner_id_list.append(inner)

game_inner_id_list = []
for game in full_games:
    inner = KNN_game_algo.trainset.to_inner_iid(game)
    game_inner_id_list.append(inner)

In [187]:
genre_inner_id_list

[11, 0, 17, 3]

In [188]:
game_inner_id_list

[8, 37, 170]

In [189]:
# Retrieve inner ids of the nearest neighbors of the game in question.
genre_neighbors_list = []
for inner in genre_inner_id_list:
    genre_neighbors = KNN_genre_algo.get_neighbors(inner, k=3)
    genre_neighbors_list.append(genre_neighbors)

game_neighbors_list = []
for inner in game_inner_id_list:
    game_neighbors = KNN_game_algo.get_neighbors(inner, k=3)
    game_neighbors_list.append(game_neighbors)

In [190]:
print(genre_neighbors_list)
print(game_neighbors_list)

[[15, 7, 1], [24, 6, 16], [14, 10, 24], [26, 24, 21]]
[[17, 35, 39], [89, 72, 144], [0, 1, 2]]


In [191]:
df = pd.DataFrame(final_genre_predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])    

In [217]:
KNN_reco_genres = df[df['uid']==streamer_name].sort_values('est', ascending = False)[['iid', 'est']]

In [202]:
df_g = pd.DataFrame(final_game_predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])   

In [216]:
KNN_reco_games = df_g[df_g['uid']==streamer_name].sort_values('est', ascending = False)[['iid', 'est']]

In [207]:
# prioritize closest neighbors to all original genres/games mentioned

genre_final_list = []
for item in genre_neighbors_list:
    genre_final_list.append(item[0])
    genre_final_list.append(item[1])

game_final_list = []
for item in game_neighbors_list:
    game_final_list.append(item[0])
    game_final_list.append(item[1])

In [208]:
print(genre_final_list)
print(game_final_list)

[15, 7, 24, 6, 14, 10, 26, 24]
[17, 35, 89, 72, 0, 1]


In [427]:
#come up with a way to weigh the most frequent neighbors in all genres/games and combine with user-recommended ones.

In [209]:
genres = [KNN_genre_algo.trainset.to_raw_iid(iiid) for iiid in set(genre_final_list)]

games = [KNN_game_algo.trainset.to_raw_iid(iiid) for iiid in set(game_final_list)]
print('The nearest neighbors of your current genres are:' + str(genres))
print('The nearest neighbors of your current games are:' + str(games))

The nearest neighbors of your current genres are:['Driving/Racing Game', 'Simulation', 'Puzzle', 'Other', 'FPS', 'Series: Souls', 'Creative']
The nearest neighbors of your current games are:['Diablo III: Reaper of Souls', 'FINAL FANTASY XIV Online', 'Marbles On Stream', 'Princess Connect! Re: Dive', 'Art', 'Battlefield V']


In [213]:
genres, games

(['Driving/Racing Game',
  'Simulation',
  'Puzzle',
  'Other',
  'FPS',
  'Series: Souls',
  'Creative'],
 ['Diablo III: Reaper of Souls',
  'FINAL FANTASY XIV Online',
  'Marbles On Stream',
  'Princess Connect! Re: Dive',
  'Art',
  'Battlefield V'])

In [210]:
# combined outputs of both methods to produce results common in both
print("We recommend you stream the following genres: " + ', '.join(item for item in set(genres + genre_user_based_list)))
print("We recommend you stream the following games: " + ', '.join(item for item in set(games + game_user_based_list)))


We recommend you stream the following genres: Driving/Racing Game, Other, Creative, Puzzle, Sports Game, Flight Simulator, Simulation, Hidden Objects, Visual Novel, Series: Souls, Gambling Game, FPS, Rhythm & Music Game
We recommend you stream the following games: FINAL FANTASY XIV Online, Battlefield V, Rocket League, The Jackbox Party Pack 3, The Elder Scrolls IV: Oblivion, Minecraft, Diablo III: Reaper of Souls, Trials Rising, Art, Marbles On Stream, Hollow Knight, Visage, Don't Starve Together, Silent Hill, Diablo, Princess Connect! Re: Dive


In [250]:
print(personal_genre_pred.sort_values('est', ascending = False))
print(personal_game_pred.sort_values('est', ascending = False))
print(KNN_reco_genres.sort_values('est', ascending = False))
print(KNN_reco_games.sort_values('est', ascending = False))

                    iid       est
11       Hidden Objects  3.366298
10        Gambling Game  2.485280
5   Driving/Racing Game  2.074401
34         Visual Novel  2.066298
9      Flight Simulator  2.032965
4              Creative  2.024459
29           Simulation  2.002190
30          Sports Game  1.923006
19                Other  1.912925
25  Rhythm & Music Game  1.905741
16                 MOBA  1.900244
13                  IRL  1.890285
6      Educational Game  1.866298
27        Series: Souls  1.862198
18           Open World  1.849337
20           Platformer  1.819807
7                   FPS  1.818362
26            Roguelike  1.816479
0                Action  1.808367
15               MMORPG  1.798344
12               Horror  1.791932
22               Puzzle  1.789946
28              Shooter  1.788344
23                  RPG  1.767695
1        Adventure Game  1.750592
8              Fighting  1.715328
2     Card & Board Game  1.709327
3           Compilation  1.677953
32            

In [252]:
frames = [personal_genre_pred, personal_game_pred, KNN_reco_genres, KNN_reco_games]

In [253]:
result = pd.concat(frames)

In [254]:
result

,iid,est
0,Action,1.808367
1,Adventure Game,1.750592
2,Card & Board Game,1.709327
3,Compilation,1.677953
4,Creative,2.024459
5,Driving/Racing Game,2.074401
6,Educational Game,1.866298
7,FPS,1.818362
8,Fighting,1.715328
9,Flight Simulator,2.032965
